# Text Summarising of the articles

## Text-Summarization
**Automatic summarization** is the process of shortening a text document with software, in order to create a summary with the major points of the original document. Technologies that can make a coherent summary take into account variables such as length, writing style and syntax.

Automatic data summarization is part of the real machine learning and data mining. The main idea of summarization is to find a subset of data which contains the "information" of the entire set. Such techniques are widely used in industry today. Search engines are an example; others include summarization of documents, image collections and videos. Document summarization tries to create a representative summary or abstract of the entire document, by finding the most informative sentences, while in image summarization the system finds the most representative and important (i.e. salient) images. For surveillance videos, one might want to extract the important events from the uneventful context.

There are two general approaches to automatic summarization: Extraction and Abstraction. 
1. *Extractive Summarization*: These methods rely on extracting several parts, such as phrases and sentences, from a piece of text and stack them together to create a summary. Therefore, identifying the right sentences for summarization is of utmost importance in an extractive method.
2. *Abstractive Summarization*: These methods use advanced NLP techniques to generate an entirely new summary. Some parts of this summary may not even appear in the original text. Such a summary might include verbal innovations. 
Research to date has focused primarily on extractive methods, which are appropriate for image collection summarization and video summarization.

In this Jupyter notebook, TextRank algorithm for extractive text summarization is implemented using Google's PageRank search algorithm to generate corelations among sentences.

### Libraries Used
- [Numpy](http://www.numpy.org)
- [Pandas](https://pandas.pydata.org/)
- [Natural Language Toolkit](https://www.nltk.org/)

### Algorithms and Concepts
- TextRank
- [PageRank](https://en.wikipedia.org/wiki/PageRank)
- [Cosine Similarity](https://en.wikipedia.org/wiki/Cosine_similarity)

### How to run
- Install the required libraries using pip, virtual environment or conda.
- Run `jupyter notebook` in your terminal.


In [3]:
# Importing the required Libraries
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt') # one time execution
import re
#nltk.download('stopwords') # one time execution
import matplotlib.pyplot as plt

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx

In [4]:
# Extract word vectors
word_embeddings = {}
file = open('glove.6B.100d.txt', encoding='utf-8')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
file.close()
len(word_embeddings)

400000

In [5]:
# reading the file
df = pd.read_excel('TASK.xlsx')

In [6]:
df

,TEST DATASET,Unnamed: 1
0,NaN,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [7]:
df.columns

Index(['TEST DATASET', 'Unnamed: 1'], dtype='object')

In [8]:
df.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)
# Deleting the first row
df.drop(0)

,TEST DATASET,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [9]:
# Converting the DataFrame into a dictionary
text_dictionary = {}
for i in range(1,len(df['TEST DATASET'])):
    text_dictionary[i] = df['Introduction'][i]
    
print(text_dictionary[1])

Acnesol Gel is an antibiotic that fights bacteria. It is used to treat acne, which appears as spots or pimples on your face, chest or back. This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor. You should normally wash and dry the affected area before applying a thin layer of the medicine. It should not be applied to broken or damaged skin. Avoid any contact with your eyes, nose, or mouth. Rinse it off with water if you accidentally get it in these areas. It may take several weeks for your symptoms to improve, but you should keep using this medicine regularly. Do not stop using it as soon as your acne starts to get better. Ask your doctor when you should stop treatment.Common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people. These are usually temporary and resolve on their own. Consult your doctor if they bother you or do not

In [10]:
# function to remove stopwords
def remove_stopwords(sen):
    stop_words = stopwords.words('english')
    
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [11]:
# function to make vectors out of the sentences
def sentence_vector_func (sentences_cleaned) : 
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)
    
    return (sentence_vector)

In [12]:
# function to get the summary of the articles

def summary_text (test_text, n = 5):
    sentences = []
    
    # tokenising the text 
    sentences.append(sent_tokenize(test_text))
    #print(sentences)
    sentences = [y for x in sentences for y in x] # flatten list
    # print(sentences)
    
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    #print(clean_sentences)

    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #print(clean_sentences)
    
    sentence_vectors = sentence_vector_func(clean_sentences)
    
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #print(sim_mat)
    
    # Finding the similarities between the sentences 
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    print(scores)
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)))
    # Extract sentences as the summary
    summarised_string = ''
    for i in range(n):
        
        try:
            summarised_string = summarised_string + str(ranked_sentences[i][1])            
        except IndexError:
            print ("Summary Not Available")
    
    return (summarised_string)

In [13]:
print("Kindly let me know in how many sentences you want the summary - ")
x = int(input())

summary_dictionary = {}

for key in text_dictionary:
    
    para = text_dictionary[key]
    print("Summary of the article - ",key)
    summary = summary_text(para,x)
    summary_dictionary[key] = summary
    
    print(summary)
    print('='*120)    
    
print ("*"*40,"The process has been completed successfully","*"*40)

Kindly let me know in how many sentences you want the summary - 
3
Summary of the article -  1
{0: 0.05117660631264973, 1: 0.0750959124816507, 2: 0.0776569205578409, 3: 0.07272925304001959, 4: 0.06896136135053554, 5: 0.06948593151107181, 6: 0.07030885149145856, 7: 0.07672434813457014, 8: 0.07525909108263469, 9: 0.0786342446736159, 10: 0.06625700471601338, 11: 0.07252305787995396, 12: 0.07891248849733594, 13: 0.06627492827064911}
Acnesol Gel is an antibiotic that fights bacteria.These are usually temporary and resolve on their own.Consult your doctor about using this medicine if you are pregnant or breastfeeding.
Summary of the article -  2


C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


{0: 0.0883798788160253, 1: 0.08900030904315201, 2: 0.09791904864823725, 3: 0.10292270933644647, 4: 0.10444658381960002, 5: 0.10548269651983565, 6: 0.10132414753213487, 7: 0.10385366630913963, 8: 0.10604495960043234, 9: 0.10062600037499625}
Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus.It works by thinning and loosens mucus in the nose, windpipe and lungs and make it easier to cough out.Ambrodil Syrup should be taken with food.For better results, it is suggested to take it at the same time every day.
Summary of the article -  3
{0: 0.08497169396695729, 1: 0.07749881612306275, 2: 0.0979738244953534, 3: 0.08347695809302506, 4: 0.09050938634409482, 5: 0.09572831680410575, 6: 0.08944049193752335, 7: 0.09720028146167708, 8: 0.09740867241828476, 9: 0.09520127965546359, 10: 0.09059027870045241}
It is used to treat infections of the lungs (e.g., pneumonia), ear, nasal sinus, urinary tract, skin and soft tissue.You should take it regularl

C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


{0: 0.06535810734156765, 1: 0.0709344935091093, 2: 0.07035634393697643, 3: 0.07809289857057329, 4: 0.07700063769098243, 5: 0.0859768005192461, 6: 0.08505149801274367, 7: 0.07322207455839258, 8: 0.08128723713043869, 9: 0.08599595344694375, 10: 0.07610216385480797, 11: 0.07704587550599024, 12: 0.07357591592222806}
Allegra 120mg Tablet belongs to a group of medicines called antihistamines.It also relieves watery eyes, runny nose, sneezing, and itching.Allegra 120mg Tablet should be taken on an empty stomach.It is used to treat various allergic conditions such as hay fever, conjunctivitis and some skin reactions such as eczema, hives and reactions to bites and stings.
Summary of the article -  7
{0: 0.06761871312275021, 1: 0.05429542145251529, 2: 0.07506350208026519, 3: 0.07293123169173334, 4: 0.07426982113442435, 5: 0.07417733114301703, 6: 0.07616570037474943, 7: 0.06721385984484368, 8: 0.07211501590076072, 9: 0.07581323419732937, 10: 0.0775206454067004, 11: 0.06992285471265795, 12: 0.070

...

C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


{0: 0.10756302328407188, 1: 0.10426799311175017, 2: 0.11033745780721332, 3: 0.11081285733635265, 4: 0.1151386192372483, 5: 0.1073253130497614, 6: 0.11559301806262526, 7: 0.11627468033878466, 8: 0.11268703777219238}
It is prescribed to treat various types of bacterial infections.Additionally, you may notice some injection site reactions like pain, swelling or redness.Ampitrust 1.5gm Injection is a combination of two antibiotics.
Summary of the article -  996
{0: 0.0471965639144301, 1: 0.05057490350575214, 2: 0.05476468438411729, 3: 0.05724064068436323, 4: 0.04543138716483015, 5: 0.05208146335986884, 6: 0.04457797423490076, 7: 0.05543490889869167, 8: 0.05484240050382855, 9: 0.05725889726853339, 10: 0.048835018009632705, 11: 0.05304955757654199, 12: 0.05400902770160387, 13: 0.05252017961960938, 14: 0.05378728078593366, 15: 0.055076209612289484, 16: 0.05318323010379605, 17: 0.05634973315812461, 18: 0.05378593951315208}
Swallow it as a whole,  do not crush, chew, break, or open them.Follow 

C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


{0: 0.0747045940009817, 1: 0.07863242973114153, 2: 0.07989636320264659, 3: 0.07474993398407824, 4: 0.0807740414315845, 5: 0.0813251707457057, 6: 0.06459130729091238, 7: 0.07794315599538458, 8: 0.07541460228618302, 9: 0.08286698952906486, 10: 0.08274888937658983, 11: 0.0681619909646004, 12: 0.07819053146112649}
Swallow the tablets whole with a drink of water.Tell your doctor if you are pregnant or breastfeeding.Arimidex 1mg Tablet  is used alone or with other treatments, such as surgery or radiation, to treat early breast cancer in postmenopausal women.
Summary of the article -  998
{0: 0.05348610360248561, 1: 0.05410214424008776, 2: 0.05775314583451181, 3: 0.053058657836883585, 4: 0.05660906110687637, 5: 0.055848894735614246, 6: 0.05451040091225101, 7: 0.05533522344239687, 8: 0.0585758458349329, 9: 0.056758935782173614, 10: 0.05560179238146307, 11: 0.05593332438030184, 12: 0.05582871375499962, 13: 0.058406769377370646, 14: 0.05766093789244428, 15: 0.04812809361245374, 16: 0.05691912526

C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
C:\Users\rasur\AppData\Local\Temp\ipykernel_23756\3253896731.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


In [14]:
summary_table = pd.DataFrame(list(summary_dictionary.items()),columns = ['TEST DATASET','Summary'])

In [15]:
data_table = pd.DataFrame(list(text_dictionary.items()),columns = ['TEST DATASET','Introduction'])

In [16]:
# Combining the findings into the table
result  = pd.concat([data_table , summary_table['Summary']], axis = 1 , sort = False)
result

,TEST DATASET,Introduction,Summary
0,1,Acnesol Gel is an antibiotic that fights bacte...,Acnesol Gel is an antibiotic that fights bacte...
1,2,Ambrodil Syrup is used for treating various re...,Ambrodil Syrup is used for treating various re...
2,3,Augmentin 625 Duo Tablet is a penicillin-type ...,It is used to treat infections of the lungs (e...
3,4,Azithral 500 Tablet is an antibiotic used to t...,These are usually temporary and subside with t...
4,5,Alkasol Oral Solution is a medicine used in th...,This will prevent you from getting an upset st...
...,...,...,...
995,996,Azapure Tablet belongs to a group of medicines...,"Swallow it as a whole, do not crush, chew, br..."
996,997,Arimidex 1mg Tablet is used alone or with oth...,Swallow the tablets whole with a drink of wate...
997,998,Arpimune ME 100mg Capsule is used to prevent y...,"However, it is not recommended while breastfee..."
998,999,Amlodac CH Tablet is a combination medicine us...,Consult your doctor If any of these bother you...


In [16]:
#result.to_csv("Summary_File.csv")